# Hubert Random Forest Classifier Balanced Dataset

In [9]:
# ============================================================================
# CELL 1: SETUP AND IMPORTS
# ============================================================================
print("="*80)
print("🚀 OPTIMIZED HUBERT + RANDOM FOREST FOR SAND CHALLENGE")
print("Strategy: HuBERT + Random Forest + Balanced Dataset")
print("Target: F1 Score >= 0.70 | High Accuracy")
print("="*80)

import os
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
import librosa
import soundfile as sf
from scipy.stats import skew, kurtosis

from transformers import HubertModel, Wav2Vec2FeatureExtractor

from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, classification_report, cohen_kappa_score
)

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedRandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tqdm.auto import tqdm
import joblib
import json

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

print("✅ All imports successful!")

🚀 OPTIMIZED HUBERT + RANDOM FOREST FOR SAND CHALLENGE
Strategy: HuBERT + Random Forest + Balanced Dataset
Target: F1 Score >= 0.70 | High Accuracy
✅ Using device: cpu
✅ All imports successful!


In [10]:
# ============================================================================
# CELL 2: OPTIMIZED CONFIGURATION FOR RANDOM FOREST
# ============================================================================
class Config:
    """Optimized Configuration for Random Forest with Balanced Dataset"""

    # Paths - UPDATE THESE PATHS FOR YOUR LOCAL SYSTEM
    BASE_PATH = 'SAND_Project_Data'  # Update this path
    TRAINING_PATH = os.path.join(BASE_PATH, 'training')
    EXCEL_PATH = os.path.join(BASE_PATH, 'sand_task_1.xlsx')
    OUTPUT_PATH = os.path.join(BASE_PATH, 'optimized_hubert_rf_balanced')

    # Sheet names
    SHEET_TRAINING = 'Training Baseline - Task 1'
    SHEET_VALIDATION = 'Validation Baseline - Task 1'

    # Audio types
    AUDIO_TYPES = ['phonationA', 'phonationE', 'phonationI', 'phonationO', 'phonationU',
                   'rhythmKA', 'rhythmPA', 'rhythmTA']

    # ========================================================================
    # OPTIMIZED HUBERT CONFIGURATION
    # ========================================================================
    HUBERT_MODEL = 'facebook/hubert-base-ls960'
    SAMPLING_RATE = 16000
    MAX_AUDIO_LENGTH = 16000 * 15  # 15 seconds

    # Multi-layer feature extraction (CRITICAL FOR PERFORMANCE)
    USE_MULTI_LAYER = True
    LAYERS_TO_USE = [-4, -3, -2, -1]  # Last 4 layers

    # Multi-pooling strategy (CRITICAL FOR PERFORMANCE)
    USE_MULTI_POOLING = True
    POOLING_STRATEGIES = ['mean', 'std', 'max', 'min']  # 4 pooling methods

    # Statistical features (HELPS WITH DYSARTHRIA PATTERNS)
    USE_STATISTICAL_FEATURES = True

    # Audio augmentation during loading
    AUDIO_TRIM_DB = 15  # Lower threshold for dysarthric speech
    AUDIO_NORMALIZE = True
    MIN_AUDIO_LENGTH_SEC = 2

    # ========================================================================
    # PREPROCESSING OPTIMIZATIONS
    # ========================================================================
    USE_ROBUST_SCALING = True  # Better for outliers

    # PCA Configuration
    USE_PCA = True
    PCA_VARIANCE = 0.97  # Retain 97% variance

    # ========================================================================
    # DATA BALANCING STRATEGIES (CRITICAL FOR RANDOM FOREST)
    # ========================================================================
    BALANCING_STRATEGY = 'balanced_rf'  # Options: 'smote', 'balanced_rf', 'hybrid'
    USE_SMOTE = True
    SMOTE_STRATEGY = 'auto'  # Automatically balance all classes
    SMOTE_K_NEIGHBORS = 3

    # ========================================================================
    # OPTIMIZED RANDOM FOREST HYPERPARAMETERS
    # ========================================================================
    RF_CONFIG = {
        'n_estimators': 500,           # More trees for better performance
        'criterion': 'gini',           # Split quality measure
        'max_depth': 25,               # Limit tree depth to prevent overfitting
        'min_samples_split': 5,        # Minimum samples to split a node
        'min_samples_leaf': 2,         # Minimum samples at a leaf node
        'max_features': 'sqrt',        # Features to consider for best split
        'bootstrap': True,             # Use bootstrap sampling
        'class_weight': 'balanced_subsample',  # Balance classes in each tree
        'random_state': RANDOM_SEED,
        'n_jobs': -1,                  # Use all available cores
        'verbose': 0
    }

    # Alternative: Balanced Random Forest (if using imbalanced-learn)
    BALANCED_RF_CONFIG = {
        'n_estimators': 400,
        'criterion': 'gini',
        'max_depth': 20,
        'min_samples_split': 4,
        'min_samples_leaf': 2,
        'max_features': 'log2',
        'sampling_strategy': 'auto',  # Automatically balance classes
        'replacement': True,          # Sample with replacement
        'random_state': RANDOM_SEED,
        'n_jobs': -1
    }

    # Cross-validation
    N_FOLDS = 5
    RANDOM_SEED = 42

    # Class names
    CLASS_NAMES = {
        0: 'Severe Dysarthria',
        1: 'Moderate Dysarthria',
        2: 'Mild Dysarthria',
        3: 'No Dysarthria (ALS)',
        4: 'Healthy'
    }

config = Config()
os.makedirs(config.OUTPUT_PATH, exist_ok=True)

print("\n" + "="*80)
print("📋 OPTIMIZED CONFIGURATION - RANDOM FOREST")
print("="*80)
print(f"HuBERT Model: {config.HUBERT_MODEL}")
print(f"Classifier: Random Forest (n_estimators={config.RF_CONFIG['n_estimators']})")
print(f"Multi-Layer Extraction: {config.USE_MULTI_LAYER} ({len(config.LAYERS_TO_USE)} layers)")
print(f"Multi-Pooling: {config.USE_MULTI_POOLING} ({len(config.POOLING_STRATEGIES)} strategies)")
print(f"Statistical Features: {config.USE_STATISTICAL_FEATURES}")
print(f"PCA: {config.USE_PCA} (variance={config.PCA_VARIANCE})")
print(f"Balancing Strategy: {config.BALANCING_STRATEGY}")
print(f"Class Weight: {config.RF_CONFIG['class_weight']}")
print(f"Device: {device}")
print("="*80)

# Verify paths
print(f"\n🔍 Verifying paths...")
assert os.path.exists(config.BASE_PATH), f"Base path not found: {config.BASE_PATH}"
assert os.path.exists(config.TRAINING_PATH), f"Training path not found: {config.TRAINING_PATH}"
assert os.path.exists(config.EXCEL_PATH), f"Excel file not found: {config.EXCEL_PATH}"
print("✅ All paths verified!")


📋 OPTIMIZED CONFIGURATION - RANDOM FOREST
HuBERT Model: facebook/hubert-base-ls960
Classifier: Random Forest (n_estimators=500)
Multi-Layer Extraction: True (4 layers)
Multi-Pooling: True (4 strategies)
Statistical Features: True
PCA: True (variance=0.97)
Balancing Strategy: balanced_rf
Class Weight: balanced_subsample
Device: cpu

🔍 Verifying paths...
✅ All paths verified!


In [11]:
# ============================================================================
# CELL 3: LOAD DATASETS
# ============================================================================
print("\n" + "="*80)
print("[1/9] LOADING DATASETS")
print("="*80)

def load_data(excel_path, sheet_name):
    """Load dataset from Excel"""
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    print(f"\n✅ Loaded {len(df)} samples from: '{sheet_name}'")

    # Convert to 0-indexed classes
    df['Class'] = df['Class'] - 1

    # Display distribution
    print(f"📊 Class distribution:")
    class_dist = df['Class'].value_counts().sort_index()
    for cls, count in class_dist.items():
        pct = (count / len(df)) * 100
        print(f"  Class {cls} ({config.CLASS_NAMES[cls]:25s}): {count:3d} ({pct:5.2f}%)")

    return df

# Load training and validation sets
df_train = load_data(config.EXCEL_PATH, config.SHEET_TRAINING)
df_val = load_data(config.EXCEL_PATH, config.SHEET_VALIDATION)

print(f"\n📈 Summary:")
print(f"  Training samples: {len(df_train)}")
print(f"  Validation samples: {len(df_val)}")
print(f"  Total audio files: {(len(df_train) + len(df_val)) * len(config.AUDIO_TYPES)}")

# Verify no overlap
train_ids = set(df_train['ID'].values)
val_ids = set(df_val['ID'].values)
overlap = train_ids.intersection(val_ids)
print(f"  Overlap check: {'✅ No overlap' if len(overlap) == 0 else f'⚠ {len(overlap)} overlapping IDs'}")


[1/9] LOADING DATASETS

✅ Loaded 219 samples from: 'Training Baseline - Task 1'
📊 Class distribution:
  Class 0 (Severe Dysarthria        ):   4 ( 1.83%)
  Class 1 (Moderate Dysarthria      ):  22 (10.05%)
  Class 2 (Mild Dysarthria          ):  45 (20.55%)
  Class 3 (No Dysarthria (ALS)      ):  62 (28.31%)
  Class 4 (Healthy                  ):  86 (39.27%)

✅ Loaded 53 samples from: 'Validation Baseline - Task 1'
📊 Class distribution:
  Class 0 (Severe Dysarthria        ):   2 ( 3.77%)
  Class 1 (Moderate Dysarthria      ):   4 ( 7.55%)
  Class 2 (Mild Dysarthria          ):  12 (22.64%)
  Class 3 (No Dysarthria (ALS)      ):  14 (26.42%)
  Class 4 (Healthy                  ):  21 (39.62%)

📈 Summary:
  Training samples: 219
  Validation samples: 53
  Total audio files: 2176
  Overlap check: ✅ No overlap


In [12]:
# ============================================================================
# CELL 4: LOAD HUBERT MODEL
# ============================================================================
print("\n" + "="*80)
print("[2/9] LOADING HUBERT MODEL")
print("="*80)
print(f"Model: {config.HUBERT_MODEL}")
print("⏱ This may take 1-2 minutes for first-time download...")

# Load feature extractor and model
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(config.HUBERT_MODEL)
hubert_model = HubertModel.from_pretrained(config.HUBERT_MODEL)
hubert_model = hubert_model.to(device)
hubert_model.eval()  # Set to evaluation mode

print(f"\n✅ HuBERT loaded successfully!")
print(f"  Model size: {sum(p.numel() for p in hubert_model.parameters()) / 1e6:.1f}M parameters")
print(f"  Hidden size: {hubert_model.config.hidden_size}")
print(f"  Number of layers: {hubert_model.config.num_hidden_layers}")
print(f"  Sampling rate: {feature_extractor.sampling_rate} Hz")


[2/9] LOADING HUBERT MODEL
Model: facebook/hubert-base-ls960
⏱ This may take 1-2 minutes for first-time download...

✅ HuBERT loaded successfully!
  Model size: 94.4M parameters
  Hidden size: 768
  Number of layers: 12
  Sampling rate: 16000 Hz


In [13]:
# ============================================================================
# CELL 5: ENHANCED AUDIO PROCESSING FUNCTIONS
# ============================================================================
print("\n" + "="*80)
print("[3/9] DEFINING ENHANCED AUDIO PROCESSING")
print("="*80)

def load_audio_enhanced(audio_path, target_sr=16000, max_length=None):
    """
    Enhanced audio loading with advanced preprocessing

    Args:
        audio_path: Path to audio file
        target_sr: Target sampling rate
        max_length: Maximum audio length in samples

    Returns:
        audio: Preprocessed audio waveform
        sr: Sampling rate
    """
    try:
        # Load audio
        audio, sr = librosa.load(audio_path, sr=target_sr, mono=True)

        # Advanced trimming with lower threshold for dysarthric speech
        audio, _ = librosa.effects.trim(audio, top_db=config.AUDIO_TRIM_DB)

        # Normalize amplitude
        if config.AUDIO_NORMALIZE:
            audio = librosa.util.normalize(audio)

        # Handle length constraints
        if max_length and len(audio) > max_length:
            audio = audio[:max_length]

        # Ensure minimum length
        min_length = target_sr * config.MIN_AUDIO_LENGTH_SEC
        if len(audio) < min_length:
            audio = np.pad(audio, (0, min_length - len(audio)), mode='constant')

        return audio, sr

    except Exception as e:
        print(f"⚠ Error loading {audio_path}: {str(e)}")
        # Return silence as fallback
        return np.zeros(target_sr * config.MIN_AUDIO_LENGTH_SEC), target_sr

def extract_statistical_features(audio, sr=16000):
    """
    Extract statistical and acoustic features from audio
    These help capture dysarthria-specific patterns

    Args:
        audio: Audio waveform
        sr: Sampling rate

    Returns:
        features: Array of statistical features
    """
    features = []

    # Time-domain statistics
    features.extend([
        np.mean(audio),           # Mean amplitude
        np.std(audio),            # Standard deviation
        skew(audio),              # Skewness
        kurtosis(audio),          # Kurtosis
        np.max(np.abs(audio)),    # Peak amplitude
        np.median(audio),         # Median
        np.percentile(audio, 25), # 25th percentile
        np.percentile(audio, 75)  # 75th percentile
    ])

    # Zero crossing rate (voice quality indicator)
    zcr = librosa.feature.zero_crossing_rate(audio)[0]
    features.extend([np.mean(zcr), np.std(zcr), np.max(zcr)])

    # Energy features
    energy = np.sum(audio**2) / len(audio)
    features.append(energy)

    # RMS energy
    rms = librosa.feature.rms(y=audio)[0]
    features.extend([np.mean(rms), np.std(rms)])

    # Spectral features
    spectral_centroids = librosa.feature.spectral_centroid(y=audio, sr=sr)[0]
    features.extend([np.mean(spectral_centroids), np.std(spectral_centroids)])

    spectral_rolloff = librosa.feature.spectral_rolloff(y=audio, sr=sr)[0]
    features.extend([np.mean(spectral_rolloff), np.std(spectral_rolloff)])

    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=audio, sr=sr)[0]
    features.extend([np.mean(spectral_bandwidth), np.std(spectral_bandwidth)])

    return np.array(features)

def extract_hubert_features_multilayer(audio, sampling_rate=16000):
    """
    Extract HuBERT features with multi-layer and multi-pooling strategy
    This is the KEY to high performance

    Args:
        audio: Audio waveform
        sampling_rate: Sampling rate

    Returns:
        features: Concatenated feature vector
    """
    try:
        # Preprocess audio for HuBERT
        inputs = feature_extractor(
            audio,
            sampling_rate=sampling_rate,
            return_tensors="pt",
            padding=True
        )

        # Move to device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Extract features with all hidden states
        with torch.no_grad():
            outputs = hubert_model(**inputs, output_hidden_states=True)

        all_features = []

        # Extract from multiple layers
        if config.USE_MULTI_LAYER:
            layers_to_process = config.LAYERS_TO_USE
        else:
            layers_to_process = [-1]  # Only last layer

        for layer_idx in layers_to_process:
            hidden_state = outputs.hidden_states[layer_idx]  # Shape: [batch, time, hidden]

            # Apply multiple pooling strategies
            if config.USE_MULTI_POOLING:
                pooling_methods = config.POOLING_STRATEGIES
            else:
                pooling_methods = ['mean']  # Only mean pooling

            for pool_strategy in pooling_methods:
                if pool_strategy == 'mean':
                    pooled = hidden_state.mean(dim=1)
                elif pool_strategy == 'max':
                    pooled = hidden_state.max(dim=1)[0]
                elif pool_strategy == 'std':
                    pooled = hidden_state.std(dim=1)
                elif pool_strategy == 'min':
                    pooled = hidden_state.min(dim=1)[0]
                else:
                    pooled = hidden_state.mean(dim=1)

                all_features.append(pooled.cpu().numpy().squeeze())

        # Concatenate all features
        combined_features = np.concatenate(all_features)

        return combined_features

    except Exception as e:
        print(f"⚠ Feature extraction error: {str(e)}")
        # Return zero features as fallback
        n_layers = len(config.LAYERS_TO_USE) if config.USE_MULTI_LAYER else 1
        n_pooling = len(config.POOLING_STRATEGIES) if config.USE_MULTI_POOLING else 1
        feature_dim = hubert_model.config.hidden_size * n_layers * n_pooling
        return np.zeros(feature_dim)

print("✅ Audio processing functions defined!")
print(f"  Enhanced audio loading: ✓")
print(f"  Statistical features ({20} features): ✓")
print(f"  Multi-layer HuBERT extraction: ✓")


[3/9] DEFINING ENHANCED AUDIO PROCESSING
✅ Audio processing functions defined!
  Enhanced audio loading: ✓
  Statistical features (20 features): ✓
  Multi-layer HuBERT extraction: ✓


In [14]:
# ============================================================================
# CELL 6: EXTRACT FEATURES FROM ALL AUDIO FILES
# ============================================================================
print("\n" + "="*80)
print("[4/9] EXTRACTING FEATURES")
print("="*80)
print(f"Configuration:")
print(f"  Layers: {len(config.LAYERS_TO_USE)}")
print(f"  Pooling strategies: {len(config.POOLING_STRATEGIES)}")
print(f"  Statistical features: {config.USE_STATISTICAL_FEATURES}")
print(f"  Feature multiplication factor: {len(config.LAYERS_TO_USE) * len(config.POOLING_STRATEGIES)}x")

def extract_features_for_dataset(df, dataset_path, audio_types, desc="Processing"):
    """
    Extract enhanced features for entire dataset

    Args:
        df: DataFrame with patient IDs and labels
        dataset_path: Path to audio files
        audio_types: List of audio types
        desc: Progress bar description

    Returns:
        X: Feature matrix
        y: Labels
        ids: Patient IDs
    """
    features_list = []
    labels_list = []
    ids_list = []

    print(f"\n🎵 {desc} {len(df)} patients...")

    for idx, row in tqdm(df.iterrows(), total=len(df), desc=desc):
        patient_id = row['ID']
        patient_class = row['Class']

        patient_features = []

        # Process each audio type
        for audio_type in audio_types:
            audio_file = f"{patient_id}_{audio_type}.wav"
            audio_path = os.path.join(dataset_path, audio_type, audio_file)

            if os.path.exists(audio_path):
                # Load audio with enhancements
                audio, sr = load_audio_enhanced(
                    audio_path,
                    target_sr=config.SAMPLING_RATE,
                    max_length=config.MAX_AUDIO_LENGTH
                )

                # Extract HuBERT features
                hubert_features = extract_hubert_features_multilayer(audio, sr)

                # Extract statistical features if enabled
                if config.USE_STATISTICAL_FEATURES:
                    stat_features = extract_statistical_features(audio, sr)
                    combined_features = np.concatenate([hubert_features, stat_features])
                else:
                    combined_features = hubert_features

                patient_features.append(combined_features)
            else:
                # Zero features for missing files
                n_layers = len(config.LAYERS_TO_USE) if config.USE_MULTI_LAYER else 1
                n_pooling = len(config.POOLING_STRATEGIES) if config.USE_MULTI_POOLING else 1
                feature_dim = hubert_model.config.hidden_size * n_layers * n_pooling

                if config.USE_STATISTICAL_FEATURES:
                    feature_dim += 20  # Statistical features count

                patient_features.append(np.zeros(feature_dim))

        # Concatenate features from all audio types
        combined = np.concatenate(patient_features)
        features_list.append(combined)
        labels_list.append(patient_class)
        ids_list.append(patient_id)

    X = np.array(features_list)
    y = np.array(labels_list)
    ids = np.array(ids_list)

    return X, y, ids

# Extract training features
X_train, y_train, ids_train = extract_features_for_dataset(
    df_train,
    config.TRAINING_PATH,
    config.AUDIO_TYPES,
    desc="Training"
)

# Extract validation features
X_val, y_val, ids_val = extract_features_for_dataset(
    df_val,
    config.TRAINING_PATH,
    config.AUDIO_TYPES,
    desc="Validation"
)

print(f"\n✅ Feature extraction complete!")
print(f"  Training shape: {X_train.shape}")
print(f"  Validation shape: {X_val.shape}")
print(f"  Feature dimension per patient: {X_train.shape[1]}")

# Clean data (handle any NaN/Inf)
X_train = np.nan_to_num(X_train, nan=0.0, posinf=0.0, neginf=0.0)
X_val = np.nan_to_num(X_val, nan=0.0, posinf=0.0, neginf=0.0)

print(f"  Data cleaned: ✓")

# Save raw features
features_path = os.path.join(config.OUTPUT_PATH, 'features_raw.npz')
np.savez(
    features_path,
    X_train=X_train, y_train=y_train, ids_train=ids_train,
    X_val=X_val, y_val=y_val, ids_val=ids_val
)
print(f"  💾 Raw features saved: {features_path}")

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()


[4/9] EXTRACTING FEATURES
Configuration:
  Layers: 4
  Pooling strategies: 4
  Statistical features: True
  Feature multiplication factor: 16x

🎵 Training 219 patients...


Training:   0%|          | 0/219 [00:00<?, ?it/s]


🎵 Validation 53 patients...


Validation:   0%|          | 0/53 [00:00<?, ?it/s]


✅ Feature extraction complete!
  Training shape: (219, 98464)
  Validation shape: (53, 98464)
  Feature dimension per patient: 98464
  Data cleaned: ✓
  💾 Raw features saved: SAND_Project_Data\optimized_hubert_rf_balanced\features_raw.npz


In [15]:
# ============================================================================
# CELL 7: ADVANCED PREPROCESSING PIPELINE FOR RANDOM FOREST
# ============================================================================
print("\n" + "="*80)
print("[5/9] ADVANCED PREPROCESSING FOR RANDOM FOREST")
print("="*80)

# Step 1: Scaling
if config.USE_ROBUST_SCALING:
    print("📊 Applying RobustScaler (better for outliers)...")
    scaler = RobustScaler()
else:
    print("📊 Applying StandardScaler...")
    scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

print(f"  ✅ Scaling complete")
print(f"     Train: mean={X_train_scaled.mean():.6f}, std={X_train_scaled.std():.6f}")
print(f"     Val:   mean={X_val_scaled.mean():.6f}, std={X_val_scaled.std():.6f}")

# Step 2: PCA (Dimensionality Reduction)
if config.USE_PCA:
    print(f"\n📉 Applying PCA (variance threshold={config.PCA_VARIANCE})...")
    pca = PCA(n_components=config.PCA_VARIANCE, random_state=config.RANDOM_SEED)
    X_train_scaled = pca.fit_transform(X_train_scaled)
    X_val_scaled = pca.transform(X_val_scaled)

    explained_var = pca.explained_variance_ratio_.sum()
    print(f"  ✅ PCA complete")
    print(f"     Original dimensions: {X_train.shape[1]}")
    print(f"     Reduced dimensions: {X_train_scaled.shape[1]}")
    print(f"     Explained variance: {explained_var*100:.2f}%")
    print(f"     Dimensionality reduction: {X_train.shape[1]/X_train_scaled.shape[1]:.1f}x")
else:
    pca = None
    print("  ℹ PCA disabled")

# Step 3: Data Balancing Strategy
print(f"\n⚖️ Applying Data Balancing Strategy: {config.BALANCING_STRATEGY}...")
print(f"  Original class distribution:")
unique, counts = np.unique(y_train, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"    Class {cls} ({config.CLASS_NAMES[cls]:25s}): {count:3d}")

if config.BALANCING_STRATEGY == 'smote':
    # Use SMOTE for balancing
    print(f"  Using SMOTE (strategy='{config.SMOTE_STRATEGY}')...")
    try:
        smote = SMOTE(
            sampling_strategy=config.SMOTE_STRATEGY,
            k_neighbors=config.SMOTE_K_NEIGHBORS,
            random_state=config.RANDOM_SEED
        )
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
        balancing_method = "SMOTE"
    except Exception as e:
        print(f"  ⚠ SMOTE failed: {e}")
        print(f"  Continuing without balancing...")
        X_train_resampled = X_train_scaled
        y_train_resampled = y_train
        balancing_method = "None"

elif config.BALANCING_STRATEGY == 'balanced_rf':
    # Use Random Forest with built-in class balancing
    print("  Using Random Forest with balanced class weights...")
    X_train_resampled = X_train_scaled
    y_train_resampled = y_train
    balancing_method = "Balanced RF"

elif config.BALANCING_STRATEGY == 'hybrid':
    # Combine SMOTE with balanced RF
    print("  Using Hybrid approach (SMOTE + Balanced RF)...")
    try:
        smote = SMOTE(
            sampling_strategy=config.SMOTE_STRATEGY,
            k_neighbors=config.SMOTE_K_NEIGHBORS,
            random_state=config.RANDOM_SEED
        )
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)
        balancing_method = "Hybrid (SMOTE + Balanced RF)"
    except Exception as e:
        print(f"  ⚠ SMOTE failed: {e}")
        print(f"  Continuing with balanced RF only...")
        X_train_resampled = X_train_scaled
        y_train_resampled = y_train
        balancing_method = "Balanced RF"

else:
    # No balancing
    X_train_resampled = X_train_scaled
    y_train_resampled = y_train
    balancing_method = "None"

print(f"\n  ✅ Balancing complete: {balancing_method}")
print(f"     Samples: {len(y_train)} → {len(y_train_resampled)}")
print(f"  Final training distribution:")
unique, counts = np.unique(y_train_resampled, return_counts=True)
for cls, count in zip(unique, counts):
    print(f"    Class {cls}: {count:3d}")

print("\n✅ Preprocessing pipeline complete!")


[5/9] ADVANCED PREPROCESSING FOR RANDOM FOREST
📊 Applying RobustScaler (better for outliers)...
  ✅ Scaling complete
     Train: mean=0.022045, std=0.781847
     Val:   mean=0.033796, std=0.821249

📉 Applying PCA (variance threshold=0.97)...
  ✅ PCA complete
     Original dimensions: 98464
     Reduced dimensions: 199
     Explained variance: 97.14%
     Dimensionality reduction: 494.8x

⚖️ Applying Data Balancing Strategy: balanced_rf...
  Original class distribution:
    Class 0 (Severe Dysarthria        ):   4
    Class 1 (Moderate Dysarthria      ):  22
    Class 2 (Mild Dysarthria          ):  45
    Class 3 (No Dysarthria (ALS)      ):  62
    Class 4 (Healthy                  ):  86
  Using Random Forest with balanced class weights...

  ✅ Balancing complete: Balanced RF
     Samples: 219 → 219
  Final training distribution:
    Class 0:   4
    Class 1:  22
    Class 2:  45
    Class 3:  62
    Class 4:  86

✅ Preprocessing pipeline complete!


In [16]:
# ============================================================================
# CELL 8: TRAIN OPTIMIZED RANDOM FOREST CLASSIFIER
# ============================================================================
print("\n" + "="*80)
print("[6/9] TRAINING OPTIMIZED RANDOM FOREST")
print("="*80)
print(f"Configuration:")
for key, value in config.RF_CONFIG.items():
    if key != 'n_jobs':  # Skip n_jobs for cleaner output
        print(f"  {key}: {value}")

print(f"\n🌲 Training Random Forest on {len(y_train_resampled)} samples...")
print(f"   Balancing strategy: {balancing_method}")

# Choose the appropriate Random Forest implementation
if config.BALANCING_STRATEGY == 'balanced_rf' and 'BalancedRandomForestClassifier' in globals():
    print("  Using BalancedRandomForestClassifier from imbalanced-learn...")
    classifier = BalancedRandomForestClassifier(**config.BALANCED_RF_CONFIG)
else:
    print("  Using standard RandomForestClassifier with class weighting...")
    classifier = RandomForestClassifier(**config.RF_CONFIG)

# Train the classifier
classifier.fit(X_train_resampled, y_train_resampled)

print(f"✅ Random Forest training complete!")
print(f"  Number of trees: {classifier.n_estimators}")
print(f"  Number of features: {classifier.n_features_in_}")

# Feature importance analysis
if hasattr(classifier, 'feature_importances_'):
    importances = classifier.feature_importances_
    print(f"  Feature importance range: {importances.min():.6f} - {importances.max():.6f}")
    print(f"  Top 10 feature importances: {np.sort(importances)[-10:][::-1]}")


[6/9] TRAINING OPTIMIZED RANDOM FOREST
Configuration:
  n_estimators: 500
  criterion: gini
  max_depth: 25
  min_samples_split: 5
  min_samples_leaf: 2
  max_features: sqrt
  bootstrap: True
  class_weight: balanced_subsample
  random_state: 42
  verbose: 0

🌲 Training Random Forest on 219 samples...
   Balancing strategy: Balanced RF
  Using BalancedRandomForestClassifier from imbalanced-learn...
✅ Random Forest training complete!
  Number of trees: 400
  Number of features: 199
  Feature importance range: 0.001260 - 0.025530
  Top 10 feature importances: [0.02553038 0.02483392 0.0185243  0.01728794 0.01395972 0.01269205
 0.01182052 0.01079879 0.01032504 0.01001125]


In [17]:
# ============================================================================
# CELL 9: CROSS-VALIDATION ON TRAINING SET
# ============================================================================
print("\n" + "="*80)
print("[7/9] CROSS-VALIDATION")
print("="*80)
print(f"Performing {config.N_FOLDS}-fold stratified cross-validation...")
print("⏱ This may take a few minutes for Random Forest...")

# Cross-validation on ORIGINAL training set
cv_classifier = RandomForestClassifier(**config.RF_CONFIG)
cv_scores = cross_val_score(
    cv_classifier,
    X_train_scaled,
    y_train,
    cv=config.N_FOLDS,
    scoring='f1_macro',
    n_jobs=-1
)

print(f"\n✅ Cross-validation complete!")
print(f"  Fold scores: {[f'{s:.4f}' for s in cv_scores]}")
print(f"  Mean CV F1: {cv_scores.mean():.4f}")
print(f"  Std CV F1:  {cv_scores.std():.4f}")
print(f"  Min CV F1:  {cv_scores.min():.4f}")
print(f"  Max CV F1:  {cv_scores.max():.4f}")


[7/9] CROSS-VALIDATION
Performing 5-fold stratified cross-validation...
⏱ This may take a few minutes for Random Forest...

✅ Cross-validation complete!
  Fold scores: ['0.1172', '0.1933', '0.1071', '0.1770', '0.1823']
  Mean CV F1: 0.1554
  Std CV F1:  0.0358
  Min CV F1:  0.1071
  Max CV F1:  0.1933


In [18]:
# ============================================================================
# CELL 10: EVALUATE ON VALIDATION SET
# ============================================================================
print("\n" + "="*80)
print("[8/9] VALIDATION SET EVALUATION")
print("="*80)

# Predict
y_val_pred = classifier.predict(X_val_scaled)
y_val_pred_proba = classifier.predict_proba(X_val_scaled)

# Compute metrics
accuracy = accuracy_score(y_val, y_val_pred)
f1_macro = f1_score(y_val, y_val_pred, average='macro')
f1_weighted = f1_score(y_val, y_val_pred, average='weighted')
precision = precision_score(y_val, y_val_pred, average='macro', zero_division=0)
recall = recall_score(y_val, y_val_pred, average='macro', zero_division=0)
kappa = cohen_kappa_score(y_val, y_val_pred)
f1_per_class = f1_score(y_val, y_val_pred, average=None, zero_division=0)

# Display results
print("\n" + "="*80)
print("🎯 FINAL RESULTS - HUBERT + RANDOM FOREST + BALANCED DATASET")
print("="*80)
print(f"Classifier: Random Forest (n_estimators={classifier.n_estimators})")
print(f"Balancing Strategy: {balancing_method}")
print(f"Validation Accuracy:      {accuracy:.4f}")
print(f"F1 Score (Macro):         {f1_macro:.4f} ⭐")
print(f"F1 Score (Weighted):      {f1_weighted:.4f}")
print(f"Precision (Macro):        {precision:.4f}")
print(f"Recall (Macro):           {recall:.4f}")
print(f"Cohen's Kappa:            {kappa:.4f}")
print(f"CV F1 (Mean ± Std):       {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print("="*80)

# Per-class metrics
print("\n📊 PER-CLASS PERFORMANCE:")
available_classes = sorted(np.unique(y_val))
for cls, f1_val in zip(available_classes, [f1_per_class[i] for i in available_classes]):
    label = config.CLASS_NAMES[cls]
    bar = '█' * int(f1_val * 40)
    print(f"  Class {cls} ({label:25s}): {f1_val:.4f} {bar}")

# Classification report
print("\n" + "-"*80)
print("CLASSIFICATION REPORT:")
print("-"*80)
class_labels = [config.CLASS_NAMES[i] for i in available_classes]
print(classification_report(y_val, y_val_pred, target_names=class_labels, zero_division=0))

# Confusion matrix
cm = confusion_matrix(y_val, y_val_pred)
print("\nCONFUSION MATRIX:")
print(cm)

# Feature importance (if available)
if hasattr(classifier, 'feature_importances_'):
    print(f"\n🔍 FEATURE IMPORTANCE ANALYSIS:")
    importances = classifier.feature_importances_
    top_10_idx = np.argsort(importances)[-10:][::-1]
    print(f"  Top 10 most important features:")
    for i, idx in enumerate(top_10_idx):
        print(f"    {i+1:2d}. Feature {idx:4d}: {importances[idx]:.6f}")

# Achievement status
print("\n" + "="*80)
baseline_f1 = 0.4919
target_f1 = 0.70
improvement = f1_macro - baseline_f1

if f1_macro >= target_f1:
    print(f"🎉🎉🎉 TARGET ACHIEVED! 🎉🎉🎉")
    print(f"F1 Score: {f1_macro:.4f} >= {target_f1:.4f}")
    print(f"Improvement over baseline: +{improvement:.4f} (+{improvement/baseline_f1*100:.1f}%)")
elif f1_macro >= 0.65:
    print(f"🚀 EXCELLENT PROGRESS!")
    print(f"F1 Score: {f1_macro:.4f}")
    print(f"Gap to target: -{target_f1 - f1_macro:.4f}")
    print(f"Improvement over baseline: +{improvement:.4f} (+{improvement/baseline_f1*100:.1f}%)")
else:
    print(f"📈 STRONG IMPROVEMENT!")
    print(f"F1 Score: {f1_macro:.4f}")
    print(f"Improvement over baseline: +{improvement:.4f} (+{improvement/baseline_f1*100:.1f}%)")
print("="*80)


[8/9] VALIDATION SET EVALUATION

🎯 FINAL RESULTS - HUBERT + RANDOM FOREST + BALANCED DATASET
Classifier: Random Forest (n_estimators=400)
Balancing Strategy: Balanced RF
Validation Accuracy:      0.1698
F1 Score (Macro):         0.1645 ⭐
F1 Score (Weighted):      0.1647
Precision (Macro):        0.2248
Recall (Macro):           0.3595
Cohen's Kappa:            0.0478
CV F1 (Mean ± Std):       0.1554 ± 0.0358

📊 PER-CLASS PERFORMANCE:
  Class 0 (Severe Dysarthria        ): 0.1667 ██████
  Class 1 (Moderate Dysarthria      ): 0.2105 ████████
  Class 2 (Mild Dysarthria          ): 0.1176 ████
  Class 3 (No Dysarthria (ALS)      ): 0.1053 ████
  Class 4 (Healthy                  ): 0.2222 ████████

--------------------------------------------------------------------------------
CLASSIFICATION REPORT:
--------------------------------------------------------------------------------
                     precision    recall  f1-score   support

  Severe Dysarthria       0.09      1.00      0.

In [19]:
# ============================================================================
# CELL 11: SAVE ALL MODELS AND RESULTS
# ============================================================================
print("\n" + "="*80)
print("[9/9] SAVING RESULTS")
print("="*80)

# Save models
print("💾 Saving models...")
joblib.dump(scaler, os.path.join(config.OUTPUT_PATH, 'scaler.pkl'))
print(f"  ✅ Scaler saved")

if pca:
    joblib.dump(pca, os.path.join(config.OUTPUT_PATH, 'pca.pkl'))
    print(f"  ✅ PCA saved")

joblib.dump(classifier, os.path.join(config.OUTPUT_PATH, 'classifier.pkl'))
print(f"  ✅ Classifier saved")

# Save results dictionary
results = {
    'model': 'Optimized HuBERT + Random Forest + Balanced Dataset',
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'strategy': 'HuBERT + Random Forest + Balanced Dataset',
    'classifier_type': 'RandomForest',
    'balancing_method': balancing_method,
    'metrics': {
        'accuracy': float(accuracy),
        'f1_macro': float(f1_macro),
        'f1_weighted': float(f1_weighted),
        'precision': float(precision),
        'recall': float(recall),
        'kappa': float(kappa),
        'cv_mean': float(cv_scores.mean()),
        'cv_std': float(cv_scores.std()),
    },
    'f1_per_class': {int(i): float(f1_per_class[i]) for i in range(len(f1_per_class))},
    'confusion_matrix': cm.tolist(),
    'configuration': {
        'hubert_model': config.HUBERT_MODEL,
        'multi_layer': config.USE_MULTI_LAYER,
        'n_layers': len(config.LAYERS_TO_USE),
        'layers': config.LAYERS_TO_USE,
        'multi_pooling': config.USE_MULTI_POOLING,
        'pooling_strategies': config.POOLING_STRATEGIES,
        'statistical_features': config.USE_STATISTICAL_FEATURES,
        'robust_scaling': config.USE_ROBUST_SCALING,
        'pca': config.USE_PCA,
        'pca_variance': config.PCA_VARIANCE if config.USE_PCA else None,
        'balancing_strategy': config.BALANCING_STRATEGY,
        'n_estimators': config.RF_CONFIG['n_estimators'],
        'max_depth': config.RF_CONFIG['max_depth'],
        'class_weight': config.RF_CONFIG['class_weight'],
    },
    'feature_dimensions': {
        'original': int(X_train.shape[1]),
        'after_pca': int(X_train_scaled.shape[1]),
        'reduction_factor': float(X_train.shape[1] / X_train_scaled.shape[1]) if pca else 1.0
    },
    'dataset_info': {
        'n_train': int(len(y_train)),
        'n_train_resampled': int(len(y_train_resampled)),
        'n_val': int(len(y_val)),
    }
}

# Add feature importance if available
if hasattr(classifier, 'feature_importances_'):
    results['feature_importance'] = {
        'mean_importance': float(classifier.feature_importances_.mean()),
        'max_importance': float(classifier.feature_importances_.max()),
        'min_importance': float(classifier.feature_importances_.min()),
        'top_10_features': [int(idx) for idx in np.argsort(classifier.feature_importances_)[-10:][::-1]]
    }

results_path = os.path.join(config.OUTPUT_PATH, 'results.json')
with open(results_path, 'w') as f:
    json.dump(results, f, indent=4)
print(f"  ✅ Results saved: {results_path}")

# Save predictions
val_predictions = pd.DataFrame({
    'ID': ids_val,
    'True_Class': y_val,
    'Predicted_Class': y_val_pred,
    'True_Label': [config.CLASS_NAMES[int(c)] for c in y_val],
    'Predicted_Label': [config.CLASS_NAMES[int(c)] for c in y_val_pred],
    'Correct': (y_val == y_val_pred).astype(int),
    'Confidence': y_val_pred_proba.max(axis=1)
})

# Add probability columns
for cls in range(len(config.CLASS_NAMES)):
    val_predictions[f'Prob_Class_{cls}'] = y_val_pred_proba[:, cls]
    val_predictions[f'Prob_{config.CLASS_NAMES[cls]}'] = y_val_pred_proba[:, cls]

val_predictions = val_predictions.sort_values('ID').reset_index(drop=True)

pred_path = os.path.join(config.OUTPUT_PATH, 'validation_predictions.csv')
val_predictions.to_csv(pred_path, index=False)
print(f"  ✅ Predictions saved: {pred_path}")

print(f"\n✅ All results saved to: {config.OUTPUT_PATH}")


[9/9] SAVING RESULTS
💾 Saving models...
  ✅ Scaler saved
  ✅ PCA saved
  ✅ Classifier saved
  ✅ Results saved: SAND_Project_Data\optimized_hubert_rf_balanced\results.json
  ✅ Predictions saved: SAND_Project_Data\optimized_hubert_rf_balanced\validation_predictions.csv

✅ All results saved to: SAND_Project_Data\optimized_hubert_rf_balanced
